## Example of backreaction setups

This notebook shows how to use the setup routine to include the effect of the dust backreaction (i.e. the dynamical feedback) into the dustpy `Simulation` object.

We also illustrate the modifications applied to the simulation object by the `setup_backreaction()` routine.


This module follows the implementation by Garate et al.([2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...871...53G%2F/abstract), [2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...635A.149G/abstract))

## Basic backreaction setup

In [1]:
from dustpy import Simulation
from dustpy import constants as c

In [2]:
sim = Simulation()

The key parameters that impact the intensity of the dust backreaction are:
* Initial global dust-to-gas ratio $\large \epsilon_0$
* Dust fragmentation velocity $\large v_\mathrm{frag}$
* Gas turbulence parameter $\large  \alpha$ 
* Dust turbulence $\large \delta_\mathrm{turb}$ (if different form $\large \alpha$)

In [3]:
sim.ini.gas.alpha = 1.e-3
sim.ini.dust.d2gRatio = 0.01
sim.ini.dust.vfrag = 1000.0

In [4]:
sim.initialize()

Afte initializing we call the `setup_backreaction() routine` that will load the backreaction coefficient updaters into the `sim`.

In [5]:
import dustpylib.dynamics

In [6]:
from dustpylib.dynamics.backreaction.setup_backreaction import setup_backreaction
setup_backreaction(sim)

Setting up the backreaction module.
Please cite the work of Garate(2019, 2020).


The setup routine now calculates the backreaction coefficients `A` and `B` which affect the gas radial and azimuthal velocities as in the follows.

$\Large v_{r, gas} = A v_\nu + 2B v_\eta$

$\Large \Delta v_{\phi, gas} = -A v_\eta + \frac{B}{2} v_\nu$


Here $v_\nu$ corresponds to the gas viscous velocity, and $v_\eta = \eta v_K$ corresponds to standard gas deviation from the keplerian speed due its own pressure support.

In a disk with a single dust species, the backreaction coefficients can be written as:

$\Large A = \frac{\epsilon +1 + \mathrm{St}^2}{(\epsilon + 1)^2 + \mathrm{St}^2}$

$\Large B = \frac{\epsilon \mathrm{St}}{(\epsilon + 1)^2 + \mathrm{St}^2}$



In a disk with low dust content and/or small particle sizes ($\epsilon \rightarrow 0, \mathrm{St}\rightarrow 0$) we have that the coefficients tend to:

$\Large A \rightarrow 1$

$\Large B \rightarrow 0$

where we recover the values for the gas evolution of a dust free disk.

In `dustpy` the backreaction coefficients are stored in the `sim.dust.backreaction` group

In [7]:
sim.dust.backreaction

Group (Backreaction coefficients)
---------------------------------
    A            : Field (Pull factor)
    B            : Field (Push factor)
  -----

The backreaction coefficients are update simultaneously when the `sim.dust.backreaction.update()` function is called.

The corresponding updater can be found in `functions_backreaction.BackreactionCoefficients()`


Now the simulation is fully setup can can be executed with `sim.run()`

### Effect on dust diffusivity

The dust content in the disk is also expected to slow down the dust diffusivity.

To implement this effect, the diffusivity is modified as follows:

$\Large D_{d} = \frac{\delta_{rad} c_s^2 \Omega_K^{-1}}{(1 + \epsilon) (1 + \mathrm{St}^2)}$

The dust diffusivity updater is included automatically in the `setup_backreaction()` routine.

The updater can be found in `functions_backreaction.dustDiffusivity_Backreaction()`

## Vertical Backreaction Setup

The previous setup assumes that the gas and dust uniformly mixed in the vertical direction, and that therefore are equally affected by the effect of the dust back-reaction.

However, we know that the dust tends to settle, and that therefore the midplane should be more intensly affected by the dust feedback than the upper layers.

To do so we assume that the gas and dust are vertically distributed following a gaussian profile:


$\Large \rho_{g, d}(z) = \frac{\Sigma_{g,d}}{\sqrt{2\pi}h_{g,d}} \exp(\frac{z^2}{2h_{g,d}^2})$

With these densities we can calculate the backreaction coefficients at every height $\large A(z), B(Z)$, and obtain a vertically weighted average to calculate the velocities with:

$\Large (\bar{A}_{g,d}, \bar{B}_{g,d}) = \frac{1}{\Sigma_{g,d}} \int \rho_{g,d}\, (A(z), B (z))\, \mathrm{d}z$


Because the gas and dust have different characteristic scale heights, this results in one pair of `A,B` backreaction coefficients for both the gas, and for each dust species.

The origin of this approach can be found in [Garate et al., 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...635A.149G/abstract) (Section 2.2.1)

To implement the vertically weighted backreaction coefficients, it is only necessary to mark the corresponding flag in the setup as follows:

`setup_backreaction(sim, vertical_setup = True)`



In [8]:
sim = Simulation()
sim.initialize()
setup_backreaction(sim, vertical_setup = True)

Setting up the backreaction module.
Please cite the work of Garate(2019, 2020).


which creates the additional fields for the new backreaction coefficients

In [9]:
sim.dust.backreaction

Group (Backreaction coefficients)
---------------------------------
    A            : Field (Pull factor (gas), accounting for dust settling)
    A_dust_se... : Field (Pull factor (dust), accounting for dust settling)
    B            : Field (Push factor (gas), accounting for dust settling)
    B_dust_se... : Field (Push factor (dust), accounting for dust settling)
  -----

`dust.backreaction.A` and `dust.backreaction.B` correspond to the effective backreaction experienced by the gas.

`dust.backreaction.A_dust_settling` and `dust.backreaction.B_dust_settling` correspond to the effective backreaction experienced by the each dust species, settling taken into account.

This setup also modifies the updater of `dust.v.rad` such that the maximum drift velocity is calculated for each dust species.

## What next?

In the standard dustpy setup, backreaction has an almost negligible effect in the coupled disk dynamics.
In `Example_BackreactionStudy.ipynb` we offer an example of how one might want to study the effect of backreaction, and a simulation setup with snowlines where the backreaction actually plays a significant role in the early disk evolution.